In [1]:
!pip install bpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 371.4/371.4 MB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 57.7 MB/s eta 0:00:00


In [2]:
# mount Google Drive to VM disk
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
import os
import bpy
import time

# dataset:
# fbx files (main dataset source)
FBX = '/content/drive/MyDrive/AutoCalibr/dataset/fbx/'
# ply (pre-converted to fix artifacts)
PLY_TO_IMPORT = '/content/drive/MyDrive/AutoCalibr/dataset/ply/'

# intermediate folders
INTERMEDIATES = '/content/intermediates/'
CONVERTED_PLY = '/content/intermediates/converted_ply/'
NORMALIZED_PLY = '/content/intermediates/normalized_ply/'

# sometimes used for debug outputting into non-cluttered directory
DIR = '/content/'

In [4]:
#@title { vertical-output: true}

!mkdir {INTERMEDIATES} 2>/dev/null
!rm -r {CONVERTED_PLY} 2>/dev/null
!mkdir {CONVERTED_PLY}

# needs to be rotated counterclockwise (left) 90 degrees - could be mostly automated (noted below)
needs_extra_rotation = ["Ace Of Spades", "Cantata-57", "Chroma Rush", "Cloudstrike", "Dead Mans Tale", "Duality", "False Promises", "Fugue 55", "Hawkmoon", "Jack Queen King 3", "Mindbenders Ambition", "No Time To Explain", "Ruinous Effigy", "Seven Seraph Carbine", "Seventh Seraph CQC-12", "Seventh Seraph Officer Revolver", "Seventh Seraph SAW", "Seventh Seraph SI-2", "Seventh Seraph VY-7", "The Fourth Horseman", "Trustee", "Witherhoard"]

for f in os.listdir(FBX):
  if f.endswith('.fbx'):
    # Isolate the name of the .fbx file (without extension)
    name = os.path.splitext(os.path.basename(f))[0]

    # Delete all mesh objects to avoid exporting multiple models into the same file
    bpy.ops.object.select_all(action='DESELECT')
    bpy.ops.object.select_by_type(type='MESH')
    bpy.ops.object.delete()

    # Load in FBX file
    bpy.ops.import_scene.fbx(filepath=os.path.join(FBX, f))

    # Select the object
    obj_object = bpy.context.selected_objects[0]
    bpy.context.view_layer.objects.active = obj_object

    # Decide on redirection
    if name in needs_extra_rotation:
      forwards_dir = 'Z'
      # Remove the file from the list as it's been accounted for
      needs_extra_rotation.remove(name)
    else:
      forwards_dir = '-X'

    # Export object to PLY
    bpy.ops.export_mesh.ply(filepath=os.path.join(CONVERTED_PLY, f.replace('.fbx', '.ply')), use_ascii=True, use_mesh_modifiers=True, use_normals=False, use_uv_coords=False, use_colors=False, axis_forward=forwards_dir, axis_up='Y')

    print()

# Ensure that any models that were supposed to receive extra rotation were hit
if len(needs_extra_rotation) > 0:
  print(f"The following models were not hit and may need manual inspection: {needs_extra_rotation}")


FBX version: 7400
Export completed '/content/intermediates/converted_ply/Coldheart.ply' in 0.433

FBX version: 7400
Export completed '/content/intermediates/converted_ply/Vigilance Wing.ply' in 0.306

FBX version: 7400
Export completed '/content/intermediates/converted_ply/Riskrunner.ply' in 0.292

FBX version: 7400
Export completed "/content/intermediates/converted_ply/Skyburner's oath.ply" in 0.077

FBX version: 7400
Export completed '/content/intermediates/converted_ply/Prometheus Lens.ply' in 0.380

FBX version: 7400
Export completed '/content/intermediates/converted_ply/Anarchy.ply' in 0.713

FBX version: 7400
Export completed '/content/intermediates/converted_ply/The Last Word.ply' in 0.339

FBX version: 7400
Export completed '/content/intermediates/converted_ply/Lord of Wolves.ply' in 0.214

FBX version: 7400
Export completed '/content/intermediates/converted_ply/Sleeper Simulant.ply' in 0.169

FBX version: 7400
Export completed '/content/intermediates/converted_ply/Huckleberry.

In [14]:
class Vertex:
  def __init__(self, x, y, z):
    self.x = x
    self.y = y
    self.z = z

  def scale(self, scale_x, scale_y, scale_z):
    self.x *= scale_x
    self.y *= scale_y
    self.z *= scale_z

  def translate(self, offset_x, offset_y, offset_z):
    self.x += offset_x
    self.y += offset_y
    self.z += offset_z

class Face:
  def __init__(self, vertices):
    self.vertices = vertices

class PlyObject:
  def __init__(self, name, vertices, faces):
    self.name = name
    self.vertices = vertices
    self.faces = faces


  def save_file(self, filename):
    with open(filename, "w") as file:
      file.write("ply\n")
      file.write("format ascii 1.0\n")
      file.write("comment Created by PlyObject class\n")
      file.write(f"element vertex {len(self.vertices)}\n")
      file.write("property float x\n")
      file.write("property float y\n")
      file.write("property float z\n")
      file.write(f"element face {len(self.faces)}\n")
      file.write("property list uchar uint vertex_indices\n")
      file.write("end_header\n")

      for vertex in self.vertices:
        file.write(f"{vertex.x} {vertex.y} {vertex.z}\n")

      for face in self.faces:
        formatted_vertices = ' '.join(str(v) for v in face.vertices)
        number_of_vertices = len(face.vertices)
        file.write(f"{number_of_vertices} {formatted_vertices}\n")


  def normalize_scale(self):
    x_coordinates = [vertex.x for vertex in self.vertices]
    y_coordinates = [vertex.y for vertex in self.vertices]
    z_coordinates = [vertex.z for vertex in self.vertices]

    max_distance = max(x_coordinates + y_coordinates + z_coordinates)
    min_distance = min(x_coordinates + y_coordinates + z_coordinates)
    normalization_range = max_distance - min_distance

    if normalization_range == 0:
      raise ValueError("Normalization range cannot be zero")

    for vertex in self.vertices:
      vertex.x = 2 * (vertex.x - min_distance) / normalization_range - 1
      vertex.y = 2 * (vertex.y - min_distance) / normalization_range - 1
      vertex.z = 2 * (vertex.z - min_distance) / normalization_range - 1


  def delete_plane(self):
    # Note: only call if the object has a plane artifact!
    max_x_index = max(range(len(self.vertices)), key = lambda index: self.vertices[index].x)
    min_x_index = min(range(len(self.vertices)), key = lambda index: self.vertices[index].x)
    max_z_index = max(range(len(self.vertices)), key = lambda index: self.vertices[index].z)
    min_z_index = min(range(len(self.vertices)), key = lambda index: self.vertices[index].z)

    indices_to_remove = set([max_x_index, min_x_index, max_z_index, min_z_index])

    for f, face in enumerate(self.faces):
      if set(face.vertices).intersection(indices_to_remove) == indices_to_remove:
        del self.faces[f]
        break
    else:
      raise ValueError("Face with all vertices not found")

    for index in sorted(indices_to_remove, reverse = True):
      del self.vertices[index]


  @classmethod
  def from_file(cls, filepath):
    with open(filepath, 'r') as f:
      if next(f).strip() != "ply":
        raise ValueError("The file being read is not a PLY file.")

      for _ in range(2):
        next(f)

      n_vertices = int(next(f).split()[-1])

      for _ in range(3):
        next(f)

      n_faces = int(next(f).split()[-1])

      for _ in range(2):
        next(f)

      vertices = []
      for _ in range(n_vertices):
        x, y, z = map(float, next(f).split())
        vertices.append(Vertex(x, y, z))

      faces = []
      for _ in range(n_faces):
        face_vertices = list(map(int, next(f).split()[1:]))
        faces.append(Face(face_vertices))

    name = os.path.splitext(os.path.basename(filepath))[0]

    return cls(name, vertices, faces)

In [6]:
#@title { vertical-output: true}

ply_objs = []

for f in os.listdir(CONVERTED_PLY):
  if f.endswith('.ply'):
    start_time = time.time()
    ply_obj = PlyObject.from_file(filepath=os.path.join(CONVERTED_PLY, f))
    ply_objs.append(ply_obj)
    elapsed_time = time.time() - start_time

    print(f'Processed object: {ply_obj.name} (took {elapsed_time:.2f} seconds)')
    print(f'\nVertice count: {len(ply_obj.vertices)}')
    print(f'Face count: {len(ply_obj.faces)}')
    print('-' * 50)


# Import pre-converted .ply files
for f in os.listdir(PLY_TO_IMPORT):
  if f.endswith('.ply'):
    start_time = time.time()
    ply_obj = PlyObject.from_file(filepath=os.path.join(PLY_TO_IMPORT, f))
    ply_objs.append(ply_obj)
    elapsed_time = time.time() - start_time

    print(f'Processed object: {ply_obj.name} (took {elapsed_time:.2f} seconds) (manually converted ply)')
    print(f'\nVertice count: {len(ply_obj.vertices)}')
    print(f'Face count: {len(ply_obj.faces)}')
    print('-' * 50)

Processed object: Transfiguration (took 0.03 seconds)

Vertice count: 6633
Face count: 7868
--------------------------------------------------
Processed object: Ruinous Effigy (took 0.14 seconds)

Vertice count: 15998
Face count: 19518
--------------------------------------------------
Processed object: Cloudstrike (took 0.31 seconds)

Vertice count: 30152
Face count: 30020
--------------------------------------------------
Processed object: 1000 Voices (took 0.22 seconds)

Vertice count: 22115
Face count: 32382
--------------------------------------------------
Processed object: Dead Mans Tale (took 0.24 seconds)

Vertice count: 24883
Face count: 29735
--------------------------------------------------
Processed object: Jade Jester (took 0.03 seconds)

Vertice count: 5949
Face count: 6295
--------------------------------------------------
Processed object: Silicon Neuroma (took 0.24 seconds)

Vertice count: 19480
Face count: 20925
--------------------------------------------------
Pro

In [7]:
#@title { vertical-output: true}

# has a giant rectangular plane originally used as a background, will need to be filtered out
has_plane_artifact = ['Abbadon', 'Blind Perdition', 'Ex Machina', 'Komodo-4FR', 'Nova Mortis', 'Trespasser', 'Vestian Dynasty', 'Vouchsafe']

# Remove plane artifact from manually specified objects
for obj in ply_objs:
  if obj.name in has_plane_artifact:
    has_plane_artifact.remove(obj.name)

    # Initialize the min and max values with the first vertex coordinates
    xmin = xmax = obj.vertices[0].x
    ymin = ymax = obj.vertices[0].y
    zmin = zmax = obj.vertices[0].z

    # Iterate over all vertices and update the min and max values
    for vertex in obj.vertices:
      xmin = min(xmin, vertex.x)
      xmax = max(xmax, vertex.x)
      ymin = min(ymin, vertex.y)
      ymax = max(ymax, vertex.y)
      zmin = min(zmin, vertex.z)
      zmax = max(zmax, vertex.z)

    xrang = xmax - xmin
    yrang = ymax - ymin
    zrang = zmax - zmin

    # Print the absolute maximum and minimum values
    print(f"Object name: {obj.name}")
    print("\nDimension Range:")
    print('X:', xrang)
    print('Y:', yrang)
    print('Z:', zrang)

    obj.delete_plane()

    # Initialize the min and max values with the first vertex coordinates
    xmin = xmax = obj.vertices[0].x
    ymin = ymax = obj.vertices[0].y
    zmin = zmax = obj.vertices[0].z

    # Iterate over all vertices and update the min and max values
    for vertex in obj.vertices:
      xmin = min(xmin, vertex.x)
      xmax = max(xmax, vertex.x)
      ymin = min(ymin, vertex.y)
      ymax = max(ymax, vertex.y)
      zmin = min(zmin, vertex.z)
      zmax = max(zmax, vertex.z)

    xrang = xmax - xmin
    yrang = ymax - ymin
    zrang = zmax - zmin

    # Print the absolute maximum and minimum values
    print("\nDimension Range (plane deleted):")
    print('X:', xrang)
    print('Y:', yrang)
    print('Z:', zrang)

    print('-' * 50)



# Ensure that any models that were supposed to receive extra rotation were hit
if len(has_plane_artifact) > 0:
  print(f"The following models were not hit and may need manual inspection: {has_plane_artifact}")

Object name: Vestian Dynasty

Dimension Range:
X: 1.831408
Y: 0.44448
Z: 3.43575

Dimension Range (plane deleted):
X: 0.9657739999999999
Y: 0.44448
Z: 2.175607
--------------------------------------------------
Object name: Nova Mortis

Dimension Range:
X: 1.831408
Y: 0.44357599999999997
Z: 3.43575

Dimension Range (plane deleted):
X: 1.031532
Y: 0.44357599999999997
Z: 2.466508
--------------------------------------------------
Object name: Trespasser

Dimension Range:
X: 1.831408
Y: 0.49324199999999996
Z: 3.43575

Dimension Range (plane deleted):
X: 0.953549
Y: 0.49324199999999996
Z: 2.055912
--------------------------------------------------
Object name: Ex Machina

Dimension Range:
X: 1.831408
Y: 0.37507999999999997
Z: 3.43575

Dimension Range (plane deleted):
X: 0.965509
Y: 0.37507999999999997
Z: 2.319017
--------------------------------------------------
Object name: Blind Perdition

Dimension Range:
X: 1.831408
Y: 0.47424900000000003
Z: 3.43575

Dimension Range (plane deleted):
X

In [8]:
#@title { vertical-output: true}
for obj in ply_objs:
  min_val = float('inf')
  max_val = float('-inf')

  for v in obj.vertices:
    min_val = min(min_val, v.x, v.y, v.z)
    max_val = max(max_val, v.x, v.y, v.z)

  print(f"Object name: {obj.name}")
  print("\nAbsolute Extrema in any Dimension:")
  print(f"Overall minimum: {min_val}")
  print(f"Overall maximum: {max_val}")

  obj.normalize_scale()

  min_val = float('inf')
  max_val = float('-inf')

  for v in obj.vertices:
    min_val = min(min_val, v.x, v.y, v.z)
    max_val = max(max_val, v.x, v.y, v.z)

  print("\nAbsolute Extrema in any Dimension (Normalized size):")
  print(f"Overall minimum: {min_val}")
  print(f"Overall maximum: {max_val}")
  print('-' * 50)

Object name: Transfiguration

Absolute Extrema in any Dimension:
Overall minimum: -0.791397
Overall maximum: 0.315061

Absolute Extrema in any Dimension (Normalized size):
Overall minimum: -1.0
Overall maximum: 1.0
--------------------------------------------------
Object name: Ruinous Effigy

Absolute Extrema in any Dimension:
Overall minimum: -0.791955
Overall maximum: 0.268863

Absolute Extrema in any Dimension (Normalized size):
Overall minimum: -1.0
Overall maximum: 1.0
--------------------------------------------------
Object name: Cloudstrike

Absolute Extrema in any Dimension:
Overall minimum: -1.191967
Overall maximum: 0.297699

Absolute Extrema in any Dimension (Normalized size):
Overall minimum: -1.0
Overall maximum: 1.0
--------------------------------------------------
Object name: 1000 Voices

Absolute Extrema in any Dimension:
Overall minimum: -0.620856
Overall maximum: 0.225458

Absolute Extrema in any Dimension (Normalized size):
Overall minimum: -1.0
Overall maximum: 

In [10]:
#@title { vertical-output: true}

count_smallest = {'x': 0, 'y': 0, 'z': 0}
count_middle = {'x': 0, 'y': 0, 'z': 0}
count_largest = {'x': 0, 'y': 0, 'z': 0}

for obj in ply_objs:
  PlyObject.normalize_scale(obj)

  min_val_x = float('inf')
  max_val_x = float('-inf')

  min_val_y = float('inf')
  max_val_y = float('-inf')

  min_val_z = float('inf')
  max_val_z = float('-inf')

  for v in obj.vertices:
    min_val_x = min(min_val_x, v.x)
    max_val_x = max(max_val_x, v.x)

    min_val_y = min(min_val_y, v.y)
    max_val_y = max(max_val_y, v.y)

    min_val_z = min(min_val_z, v.z)
    max_val_z = max(max_val_z, v.z)

  ranges = {
    'x': max_val_x - min_val_x,
    'y': max_val_y - min_val_y,
    'z': max_val_z - min_val_z
  }
  sorted_ranges = sorted(ranges.items(), key=lambda item: item[1])

  print(f"Object name: {obj.name}")
  print(f"\nRange in x-dimension: {ranges['x']}")
  print(f"Range in y-dimension: {ranges['y']}")
  print(f"Range in z-dimension: {ranges['z']}")
  print('-' * 50)

  count_smallest[sorted_ranges[0][0]] += 1
  count_middle[sorted_ranges[1][0]] += 1
  count_largest[sorted_ranges[2][0]] += 1

print("\nTimes dimension had largest dimensional range:")
print(f"X: {count_largest['x']}")
print(f"Y: {count_largest['y']}")
print(f"Z: {count_largest['z']}")

print("\nTimes dimension had middle dimensional range:")
print(f"X: {count_middle['x']}")
print(f"Y: {count_middle['y']}")
print(f"Z: {count_middle['z']}")

print("\nTimes dimension had smallest dimensional range:")
print(f"X: {count_smallest['x']}")
print(f"Y: {count_smallest['y']}")
print(f"Z: {count_smallest['z']}")

# manually rotated objects so this doesn't matter, but I would use this to auto-rotate objects if using a large dataset

Object name: Transfiguration

Range in x-dimension: 0.12668533283685446
Range in y-dimension: 0.551764278445273
Range in z-dimension: 2.0
--------------------------------------------------
Object name: Ruinous Effigy

Range in x-dimension: 0.19434059376820545
Range in y-dimension: 0.5658501269774832
Range in z-dimension: 2.0
--------------------------------------------------
Object name: Cloudstrike

Range in x-dimension: 0.11114840507872237
Range in y-dimension: 0.3806061224462396
Range in z-dimension: 2.0
--------------------------------------------------
Object name: 1000 Voices

Range in x-dimension: 0.3734689488771308
Range in y-dimension: 0.8419380986253331
Range in z-dimension: 1.9906441344465529
--------------------------------------------------
Object name: Dead Mans Tale

Range in x-dimension: 0.12042738034440625
Range in y-dimension: 0.4509729100079758
Range in z-dimension: 2.0
--------------------------------------------------
Object name: Jade Jester

Range in x-dimension:

In [12]:
#@title { vertical-output: true}

!rm -r {NORMALIZED_PLY} 2>/dev/null
!mkdir {NORMALIZED_PLY}

for obj in ply_objs:
  start_time = time.time()
  obj.save_file(NORMALIZED_PLY + obj.name + '.ply')
  elapsed_time = time.time() - start_time

  print(f'Exported object: {obj.name} (took {elapsed_time:.2f} seconds)\n')

Exported object: Transfiguration (took 0.04 seconds)

Exported object: Ruinous Effigy (took 0.08 seconds)

Exported object: Cloudstrike (took 0.15 seconds)

Exported object: 1000 Voices (took 0.13 seconds)

Exported object: Dead Mans Tale (took 0.16 seconds)

Exported object: Jade Jester (took 0.03 seconds)

Exported object: Silicon Neuroma (took 0.09 seconds)

Exported object: Prometheus Lens (took 0.07 seconds)

Exported object: Lumina (took 0.07 seconds)

Exported object: Bastion (took 0.11 seconds)

Exported object: Redrix's Claymore (took 0.06 seconds)

Exported object: Pluperfect (took 0.09 seconds)

Exported object: Ikelos_SG_V1.0.1 (took 0.11 seconds)

Exported object: Blasphemer (took 0.07 seconds)

Exported object: Garden Progeny (took 0.12 seconds)

Exported object: Austringer (took 0.07 seconds)

Exported object: Ikelos SR v1.0.1 (took 0.11 seconds)

Exported object: Vestian Dynasty (took 0.06 seconds)

Exported object: Red Death (took 0.06 seconds)

Exported object: Crimso